# Setup

In [ ]:
! pip install datasets transformers torch
! pip install sentencepiece
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch, sentencepiece
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available")

GPU available: Tesla T4


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Training

In [ ]:
def get_question(text, premise_column):
    return text.replace("[","").replace("]", "").split("<")[0].replace(f"{premise_column}: ", "")

def get_answer(text, hypothesis_column):
    #print(text)
    return text.replace("[","").replace("]", "").split(">")[1].replace(f"{hypothesis_column}: ", "")

def make_dataset(file_name, dataset_type):
    if dataset_type=="mrpc" or dataset_type=="rte":
      premise_column = "Sentence1"
      hypothesis_column = "Sentence2"
    elif dataset_type=="mnli":
      premise_column = "Premise"
      hypothesis_column = "Hypothesis"
    elif dataset_type=="qnli":
      premise_column = "Question"
      hypothesis_column = "Sentence"

    df = pd.read_csv(file_name)

    df["original_question"] = df["original_text"].apply(lambda x: get_question(x, premise_column))
    df["original_answer"] = df["original_text"].apply(lambda x: get_answer(x, hypothesis_column))
    df["perturbed_question"] = df["perturbed_text"].apply(lambda x: get_question(x, premise_column))
    df["perturbed_answer"] = df["perturbed_text"].apply(lambda x: get_answer(x, hypothesis_column))

    df_og = df[["original_question", "original_answer", "ground_truth_output"]]
    df_adv = df[["perturbed_question", "perturbed_answer", "ground_truth_output"]]
    df_og = df_og.rename(columns = {"original_question": "question", "original_answer": "sentence", "ground_truth_output": "label"})
    df_adv = df_adv.rename(columns = {"perturbed_question": "question", "perturbed_answer": "sentence", "ground_truth_output": "label"})

    df_merged = pd.concat([df_og, df_adv], ignore_index=True)
    df_merged.reset_index(names="idx")

    train, validation = train_test_split(df_merged, test_size=0.2)


    train = Dataset.from_pandas(train)
    validation = Dataset.from_pandas(validation)
    my_adv_dataset = DatasetDict({
        "train": train,
        "validation": validation
    })
    return my_adv_dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["question"], examples["sentence"], truncation=True, padding="max_length", max_length=128)

def load_model_and_tokenizer(save_directory):
    model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
    tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)
    return model, tokenizer

def train_model(model, tokenizer, dataset, adv_save_directory):
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    training_args = TrainingArguments(
        output_dir="./results/mrpc",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        num_train_epochs=2,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer
    )

    trainer.train()

    model.save_pretrained(adv_save_directory)
    tokenizer.save_pretrained(adv_save_directory)

# Evaluation

In [ ]:
def evaluate_model(model, tokenizer, validation_set, device):
    model.eval()
    preds = []

    with torch.no_grad():
        for item in validation_set:
            inputs = tokenizer(item['perturbed_question'], item['perturbed_answer'], return_tensors="pt", truncation=True, padding=True).to(device)
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=1)
            preds.append(predictions.cpu().item())

    df = pd.DataFrame(validation_set)
    df["evaluation_output"] = preds
    return df

def evaluate_model_on_dataset(val_file, model_save_directory, dataset_type, log_save_file):
    if dataset_type=="mrpc" or dataset_type=="rte":
        premise_column = "Sentence1"
        hypothesis_column = "Sentence2"
    elif dataset_type=="mnli":
        premise_column = "Premise"
        hypothesis_column = "Hypothesis"
    elif dataset_type=="qnli":
        premise_column = "Question"
        hypothesis_column = "Sentence"
    df_val = pd.read_csv(val_file)
    df_val["perturbed_question"] = df_val["perturbed_text"].apply(lambda x: get_question(x, premise_column))
    df_val["perturbed_answer"] = df_val["perturbed_text"].apply(lambda x: get_answer(x, hypothesis_column))
    attacked_ds = DatasetDict({
        "validation": Dataset.from_pandas(df_val)
    })
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = DebertaV2ForSequenceClassification.from_pretrained(model_save_directory).to(device)
    tokenizer = DebertaV2Tokenizer.from_pretrained(model_save_directory)

    df = evaluate_model(model, tokenizer, attacked_ds["validation"], device)
    df.to_csv(log_save_file)
    model_perf = (df["ground_truth_output"] == df["evaluation_output"]).sum()
    return model_perf


# MRPC Deberta

## TextBugger

In [ ]:
dataset_type = "mrpc"
attack_name = "textbugger"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.269082
2,No log,0.299831


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 28 / 200
Number of correct outputs after training: 154 / 200


## TextFooler

In [ ]:
dataset_type = "mrpc"
attack_name = "textfooler"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.303892
2,No log,0.269843


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 20 / 200
Number of correct outputs after training: 151 / 200


## DeepWordBug

In [ ]:
dataset_type = "mrpc"
attack_name = "DeepWordBug"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.304106
2,No log,0.337711


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 24 / 200
Number of correct outputs after training: 156 / 200


## PWWS

In [ ]:
dataset_type = "mrpc"
attack_name = "PWWS"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.308454
2,No log,0.383853


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 19 / 200
Number of correct outputs after training: 153 / 200


# RTE Deberta

## TextBugger

In [ ]:
dataset_type = "rte"
attack_name = "textbugger"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch,Training Loss,Validation Loss
1,No log,0.462197
2,No log,0.544932


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 15 / 200
Number of correct outputs after training: 132 / 200


## TextFooler

In [ ]:
dataset_type = "rte"
attack_name = "textfooler"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch,Training Loss,Validation Loss
1,No log,0.495025
2,No log,0.537928


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 6 / 200
Number of correct outputs after training: 128 / 200


## DeepWordBug

In [ ]:
dataset_type = "rte"
attack_name = "DeepWordBug"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch,Training Loss,Validation Loss
1,No log,0.474934
2,No log,0.603686


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 16 / 200
Number of correct outputs after training: 133 / 200


## PWWS

In [ ]:
dataset_type = "rte"
attack_name = "PWWS"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch,Training Loss,Validation Loss
1,No log,0.533798
2,No log,0.626014


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 11 / 200
Number of correct outputs after training: 110 / 200


# MNLI Deberta

## TextBugger

In [ ]:
dataset_type = "mnli"
attack_name = "textbugger"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.527004
2,No log,0.548181


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 13 / 200
Number of correct outputs after training: 116 / 200


## TextFooler

In [ ]:
dataset_type = "mnli"
attack_name = "textfooler"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.543488
2,No log,0.536206


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 18 / 200
Number of correct outputs after training: 94 / 200


## DeepWordBug

In [ ]:
dataset_type = "mnli"
attack_name = "DeepWordBug"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.617179
2,No log,0.608760


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 12 / 200
Number of correct outputs after training: 106 / 200


## PWWS

In [ ]:
dataset_type = "mnli"
attack_name = "PWWS"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.593907
2,No log,0.612723


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 8 / 200
Number of correct outputs after training: 81 / 200


# QNLI Deberta

## TextBugger

In [ ]:
dataset_type = "qnli"
attack_name = "textbugger"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Epoch,Training Loss,Validation Loss
1,No log,0.422865
2,No log,0.483909


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 79 / 200
Number of correct outputs after training: 143 / 200


## TextFooler

In [ ]:
dataset_type = "qnli"
attack_name = "textfooler"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.400281
2,No log,0.398989


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 79 / 200
Number of correct outputs after training: 141 / 200


## DeepWordBug

In [ ]:
dataset_type = "qnli"
attack_name = "DeepWordBug"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.394062
2,No log,0.421483


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 66 / 200
Number of correct outputs after training: 134 / 200


## PWWS

In [ ]:
dataset_type = "qnli"
attack_name = "PWWS"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/{dataset_type}_trainlog_{attack_name}.csv"
save_directory = f"/content/drive/My Drive/finetuned_models/{dataset_type}-deberta-xsmall"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{dataset_type}_{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/{dataset_type}_vallog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/{dataset_type}_vallog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name, dataset_type)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, dataset_type, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, dataset_type, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.415181
2,No log,0.414835


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of correct outputs before training: 74 / 200
Number of correct outputs after training: 141 / 200
